### Convert model to TFLite model

In [7]:
# Robust .keras -> .tflite converter (no SavedModel route)
# Works from tools/converter_notebook.ipynb

from pathlib import Path
import tensorflow as tf

# --- Paths (relative to the notebook in tools/) ---
model_path  = Path("../outputs/step_2/models/step_2_bs32_k3_do0.3_act-elu_opt-adamax_seed27.keras").resolve()
tflite_dir  = Path("../outputs/step_2/tflite").resolve()
tflite_dir.mkdir(parents=True, exist_ok=True)
tflite_path = tflite_dir / "step_2_bs32_k3_do0.3_act-elu_opt-adamax_seed27.tflite"

# --- Load model ---
assert model_path.exists(), f"Model not found at: {model_path}"
model = tf.keras.models.load_model(str(model_path))

# --- Ensure the model is 'built' with a concrete input shape ---
# Infer shape from the model's first input
inp = model.inputs[0]
shape = list(inp.shape)
# Replace None batch with 1
shape[0] = 1
dummy = tf.zeros(shape, dtype=inp.dtype or tf.float32)
# One forward pass to finalize shapes
_ = model(dummy)

def convert_with_keras_model(m):
    converter = tf.lite.TFLiteConverter.from_keras_model(m)
    # Optional size reduction:
    # converter.optimizations = [tf.lite.Optimize.DEFAULT]
    return converter.convert()

def convert_with_concrete_fn(m):
    @tf.function
    def serve(x):
        return m(x)
    concrete = serve.get_concrete_function(tf.TensorSpec(shape, dummy.dtype))
    converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete])
    # Optional size reduction:
    # converter.optimizations = [tf.lite.Optimize.DEFAULT]
    return converter.convert()

# --- Try direct Keras path; fallback to concrete function if needed ---
try:
    tflite_bytes = convert_with_keras_model(model)
except Exception as e1:
    print("Direct Keras conversion failed, retrying with concrete function...\n", repr(e1))
    tflite_bytes = convert_with_concrete_fn(model)

tflite_path.write_bytes(tflite_bytes)
print("✅ TFLite saved to:", tflite_path)


Direct Keras conversion failed, retrying with concrete function...
 AttributeError("'Sequential' object has no attribute '_get_save_spec'")


2025-10-12 12:18:27.005948: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2025-10-12 12:18:27.006353: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-10-12 12:18:27.006781: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-10-12 12:18:27.006789: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
W0000 00:00:1760267907.157350 9900951 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1760267907.157722 9900951 tf_tfl_flatbuffer_helpers.cc:393] Ignored 

✅ TFLite saved to: /Users/marcelldemeter/GIT/CodeInstitute/ci-p5-mildew-detector/outputs/step_2/tflite/step_2_bs32_k3_do0.3_act-elu_opt-adamax_seed27.tflite
